# Text Technologies for Data Science (INFR11145)

## Definitions of Information Retrieval

*Given query $Q$, find relevant documents $D$.*

**Main issues:**
- Effectiveness
    - need to find relevant documents
    - different from relational DBs
    
|&nbsp;|IR|DB|
|---|---|---|
|**Retrieving**|*Unstructured* data, free text with metadata|*Stuctured* data, clear semantics based on formal model|
|**Queries**|Natural language, boolean|Formally-defined (SQL), unambiguous.|
|**Result**|Imprecise (need to measure relevance)|Exact|
|**Interaction with system**|Important|One-shot queries|

- Efficiency
    - need to find them quickly
    - data constantly changes, need to keep up

**Components:**
- Documents
    - *element* to be retrieved
    - unstructured nature
    - unique ID
    - $N$ documents $\rightarrow$ Collection
- Queries
    - text to express *information need*
    - same information needs - multiple queries: North Carolina storm vs Florence
    - same query - multiple information needs: Apple
    - different forms: keywords, sample image, tune
- Relevant documents
    - similar vocabulary $\rightarrow$ similar meaning 
    - similarity: string match, word overlap, retrieval model $P(D|Q)$
    - challenges: not clear semantics (Shakespeare), inherent ambiguity (e.g. in queries), highly subjective

**Bag-of-words:**
- Re-ordering doesn't destroy topic
- negations lost
- not work for all languages (Chinese, images, music)

**Systems perspective:**
- Indexing process (offline)
- Search (retrieval) process (online)

## Laws of Text

### Zipf's Law

$$r \times P_r \cong \text{const} \quad \Leftrightarrow \quad P_r \cong \frac{\text{const}}{r}$$

- $r$: rank of term according to frequency
- $P_r$: probability of appearance of term
- Reciprocal proportion
- $\approx$ 50% terms appears once

### Benford's Law

$$P(d) = \log (1 + \frac{1}{d})$$

- $d$: first digit of frequency
- $P(d)$: pmf of $d$

### Heap's Law

$$v(n) = k \times n^b$$

- $n$: number of words
- $v$: number of unique words in the $n$ words
- $b$: typically $\in (0.4,0.7)$
- Accurate for most collections with different $k$ and $b$
- Not ccurate when $n$ is small
- Not saturated because of spelling errors, names, emails, codes, etc. 

### Contagion

- Most words do not appear that much
- Once you see a word $\rightarrow$ expect to ee again
- Like rare contagious disease
- Terms appearing twice close to each other

## Preprocessing

*Identify the optimal form of the term to be indexed to achieve the best retrieval performance.*

### Tokenisation
- Split at non-letter characters
- For French, abstract
- For German, compound splitter
- For Chinese or Japanese, segmentation
- Special setup in some applications (e.g. hashtags in social media)


### Stopping
- Stop words: the most common words (e.g. the, a, is)
- $\approx$ 30%~40% of text
- Less influence on topic
- New stop words in specific domains (e.g. 'RT' in tweets)
- Trend to keep stop words in web search
    - good compression techniques
    - good query optimisation techniques
    - probabilistic retrieval models give low wait

### Normalisation
- Case folding
- Accents removal (e.g. café)
- Equivalence classes (e.g. Ph.D. vs PhD)
- Be consistent between documents and queries

### Stemming
- Reduce morphological variations of words
    - inflectional (plurals, tenses)
    - derivational (verbs nouns)
- Basic types
    - Dictionary-based: use lists of related words
    - Algorithmic
        - remove suffixes in English
        - but false negatives or false positives exist
- Usually achieve 5%~10% retrieval effectiveness improvement in English
- Even higher in e.g. Fininsh or Arabic

### Limitations
- Irregular verbs (e.g. saw vs see)
- Different spellings (e.g. colour vs color)
- Synonyms (e.g. car vs vehicle)
- Solution: query expansion
    - more vocabulary, longer query
    - potentially more powerful, but less efficient

## Indexing

**Document vectors:**
- Each vector is a document
- Each value is frequency/binary of each term
- Collection matrix: all vectors
![](document_vectors.png)
(Adpated from [Lecture 5: Indexing (1) - Text Technologies for Data Science)](https://www.inf.ed.ac.uk/teaching/courses/tts/handouts2018/05Indexing.pdf)

**Term vectors:**
- Each vector is a term
- Each value is frequency/binary in each document
- Transpose of collection matrix
![](inverted_vectors.png)
(Adpated from [Lecture 5: Indexing (1) - Text Technologies for Data Science)](https://www.inf.ed.ac.uk/teaching/courses/tts/handouts2018/05Indexing.pdf)

**Collection matrix:**
- Terms against documents
- Extremely sparse

### Inverted index
- Sparse representation of collection matrix
- Construction
    1. Term sequence
    1. Sorting by term then by docID
    1. Posting
- Inverted index
```python
{term: list(docID)}
```
- Inverted index with frequency
```python
{term: list({docID: count})}
```
- Proximity index
```python
{term: list(tuple(docID, term pos))}
```

### Searching 

#### Boolean search
- Boolean: exist / not-exist
- Multiword search: logical operators (AND, OR, NOT)
- Query processing: linear merge $O(n)$
```python
for i in set(index[term1] + index[term2]):
    if i in index[term1] and i in index[term2]:
        print i
```

#### Phrase search
- Bi-gram index
    - index size will explode
- Use proximity search (with proximity 1)

#### Proximity search
- `#3(term1, term2)`
- Query processing: linear merge + check terms positions

### Extent index

- Special term (e.g. link)
```python
{link: list(tuple(docID, start, end))}
```
- Special field (e.g. headline)
```python
{headline: list(tuple(docID, start, end))}
{term: list(tuple(docID, pos+(end-start)))}
```

### Index compression

- Delta encoding: store ID using delta (difference) from the previous ID
- v-byte encoding
    - variable byte storage
    - high bit is <span style="color:red">1</span>: read following 7 bits
    - high bit is <span style="color:red">0</span>: read following 7 bits and check next high bit
    - examples
        - 6: <span style="color:red">1</span>0000110
        - 129: <span style="color:red">0</span>0000001<span style="color:red">1</span>0000001
- more compression = less storage = more processing

### Dictionary data structures

#### Hashes
- Hase each term to a integer
- Pro: lookup is faster than tree - $O(1)$
- Cons: 
    - no easy way to find minor variants
    - no prefix search
    - need to rehasing everything occasionally as vocabulary grows
    
#### B-tree
- Pros:
    - solve the prefix problem
    - mitigate the rebalacing problem of binary search tree
- Cons: slower - $O(\log M)$

### Permuterm indexes

- Add \$ at the end of the term and index all permutations
- Add \$ at the end of wild-card query and permutate until \* occurs at the end
- Look up term

Example: `hello`
- Indexing: `hello$`, `ello$h`, `llo$he`, `lo$hel`, `o$hell`, `$hello`
- Query: `he*o` $\rightarrow$ `he*o$` $\rightarrow$ `o$he*`

### Character n-gram indexes

Example: `monkey`, `moon` (bigrams)
- Indexing: `$m`, `mo`, `on`, `nk`, `ke`, `ey`, `y$`, `$m`, `mo`, `oo`, `on`, `n$`
- Query: `mon*`


1. Transform query to `$m`, `mo`, `on`
1. Find possible terms: `monkey`, `moon`
1. Post-filter: eliminate `moon`
1. Look up surviving terms `monkey` in document

**Application: spelling correction**
- E.g. OCR
- Possible corrections = most matching results
- E.g. elepgant $\rightarrow$ elegant OR elephant

## Ranked Retrieval

### Boolean retrieval

- Pros: good for expert users with precise understanding of needs (e.g. patent search)
- Cons: 
    - write Boolean queries
    - need to go through thousands of results

### Jaccard coeffecient

$$\operatorname{jaccard}(A,B) = \frac{|A \cap B|}{|A \cup B|}$$

- Don't consider term frequency
- Treat all terms equally
- Driven by length

### TFIDF

\begin{align}
\operatorname{df}(t) &= \sum_{i=1}^N \mathbb 1(t \in d_i) \\
\operatorname{idf}(t) &= \log_{10} \frac{N}{\operatorname{df}(t)} \\
\operatorname{tf}(t,d_i) &= \#\{t \in d_i\} \\
w_{t,d_i} &= \left( 1 + \log_{10} \operatorname{tf}(t,d_i) \right) \times \operatorname{idf}(t) \\
\operatorname{score}(q,d_i) &= \sum_{t \in q \cap d_i} w_{t,d_i}
\end{align}

- $\operatorname{idf}$ (inverse document frequency) measures the importance of a term $t$ in a collection $\{d_i\}$
- $\operatorname{tf}$ (term frequency) measures the importance of a term $t$ in a document $d$
- $\operatorname{cf}$ (collection frequency, $\displaystyle \sum_{i=1}^N \# \{t \in d_i\}$) is less commonly used in IR
- SMART notation: `ddd.qqq`
    - Example: `lnc.ltc`
        - For documents: logarithm of tf, no df (1), cosine normalisation
        - For queries: logarithm of tf, inverse df, cosine normalisation
    - Table of acronyms
    
|Term Frequency|Document Frequency|Normalisation|
|---|---|---|
|`n`: natural|`n`: no (1)|`n`: none (1)|
|`l`: logarithm|`t`: idf|`c`: cosine|
|`a`: augmented|`p`: prob idf (like ReLU)|`u`: pivoted unqiue|
|`b`: boolean|&nbsp;|`b`: byte size|
|`L`: log ave|&nbsp;|&nbsp;|

### Vector space model (VSM)

\begin{align}
M &= \#\{q \cap d_i\} \\
\mathbf q &= (w_{t_1, q}, w_{t_2,q}, \ldots, w_{t_M, q})^\top \\
\mathbf d_i &= (w_{t_1, d_i}, w_{t_2, d_i}, \ldots, w_{t_M, d_i})^\top \\
\operatorname{score}(q, d_i) &= \cos(\mathbf q, \mathbf d_i) = \frac{\mathbf q^\top \mathbf d_i}{\|\mathbf q\|_2 \|\mathbf d_i\|_2} = \frac{\sum_{t \in q \cap d_i} w_{t,d_i} w_{t,q}}{\sqrt{\sum_{j=1}^M w_{t_j,q}^2} \sqrt{\sum_{j=1}^M w_{t_j,d_i}^2}}
\end{align}

- Heuristic
- No notion of relevance
- Any weighting scheme, similarity measure can be used
- Components are not interpretable

### Probability ranking principle (PRP)

- Rank docs by probability of relevance: $P(R|D_{r_1}) > P(R|D_{r_2}) > \cdots$
- Estimate probability as accurate as possible: $\hat P(R|D) \approx P(R|D)$
- Estimate with all possibly available data: $\hat P(R|D, Q, \text{Session}, \text{Context}, \text{User profile}, \ldots)$
- Classification problem: $D$ is relevant if $P(R=0|D) > P(R=1|D)$

### Okapi BM25 model

$$w_{t,d_i} = \frac{\operatorname{tf}(t,d_i)}{k \frac{L_{d_i}}{\bar L} + \operatorname{tf}(t,d_i) + 0.5} \times \log_{10} \frac{N - \operatorname{df}(t) + 0.5}{\operatorname{df}(t) + 0.5}$$

- Best practices $k=1.5$
- $\displaystyle \bar L = \frac{1}{N} \sum_{i=1}^N L_{d_i}$

**Assumptions:**
- Binary features (term occurrence)
- Terms are independence (Naive Bayes)

### Language model (LM)

- The likelihood $P(q|M_d)$ of generating query $q=\{t_1, \ldots, t_K\}$ from language model $M_d$
- Rank docs based on posterior $P(d|q) \propto P(q|M_d) P(d)$
- Prior $P(d)$ often chosen to be Uniform

|Model|Likelihood|
|---|---|
|Unigram LM|$$P(q|M_d) = \prod_{k=1}^K P(t_k|M_d) = \prod_{\text{unique }t} P(t|M_d)^{\operatorname{tf}(t,q)}$$|
|Stochastic LM|$$P(q|M_d) = P(t_1|M_d) \prod_{k=2}^K P(t_k|M_d, t_{k-1}, \ldots, t_1)$$|
|Bigram LM|$$P(q|M_d) = P(t_1|M_d) \prod_{k=2}^K P(t_k|M_d, t_{k-1})$$|

- MLE $\displaystyle \hat P(t|M_d) = \frac{\operatorname{tf}(t,d)}{L_d}$ (0 if term not occured in $d$)

**Mixture model (Jelinek-mercer smoothing):**

$$P(t|d) = \lambda P(t|M_d) + (1-\lambda) P(t|M_c)$$

- Large $\lambda$: 'conjunctive-like' search - tends to retrieve docs containing all query words
- Small $\lambda$: more disjunctive, suitable for long queries

## Evaluation

|&nbsp;|relevant|irrelevant|
|---|---|---|
|**retrieved**|TP|FP|
|**not retrieved**|FN|TN|

### Without ranking

|Measure|Formula|Notes|
|---|---|---|
|Precision|$$P=\frac{\text{TP}}{\text{TP}+\text{FP}}$$|High precision may miss many (less) relevant docs|
|Recall|$$R=\frac{\text{TP}}{\text{TP}+\text{FN}}$$|High recall may contain many irrelevant docs|
|Accuracy|$$A=\frac{\text{TP}+\text{TN}}{\text{TP}+\text{FP}+\text{TN}+\text{FN}}$$|TN dominates the result if #irrelevant >> #relevant|
|Balanced F-measure|$$F_1=\frac{2 \cdot P \cdot R}{P+R}$$|Harmonic mean of precision and recall. Emphasises the importance of small values|
|General F-measure|$$F_\beta=\frac{(\beta^2+1) \cdot P \cdot R}{\beta^2P + R}$$|Controls the importance of recall (than precision)|

### With ranking (binary relevance)

- Precision @ K (P@K): Cut-off ranked list at rank K then calculate precision
    - (average badly)
- R-Precision: P@R where R is the number of known relevant docs
    - R is different given different query
    - (not realistic)
- Average Precision (AP): $\displaystyle \operatorname{AP}(q)=\frac{1}{\#\mathrm{Rel}_q} \sum_{k=1}^{n} P(k) \mathbb 1(d_k \in \mathrm{Rel}_q)$
    - Expectation of P@K where K is uniformly distributed
- Mean Average Precision (MAP): $\displaystyle \operatorname{MAP} = \frac{1}{Q} \sum_{q=1}^Q \operatorname{AP}(q)$
    - A mix between precision and recall
    - Highly focus on finding relevant docs as early as possible
    - Most commonly used for most IR tasks

### Graded relevance

|Measure|Formula|
|---|---|
|Gain|$G(k)=$ grade of retrieved doc at rank $k$|
|Discounted Gain|$$\operatorname{DG}(k) = \begin{cases}
\operatorname{G}(k) & k=1 \\
\displaystyle \frac{\operatorname{G}(k)}{\log_2 k} & k \geq 2
\end{cases}$$|
|Discounted Cumulative Gain|$$\operatorname{DCG}(k) = \sum_{i=1}^k \operatorname{DG}(k)$$|
|Ideal Gain|$$\operatorname{iG}(k) = G_{(k)}$$ (should include all relevant docs)|
|Ideal DG|$$\operatorname{iDG}(k) = \begin{cases}
\operatorname{iG}(k) & k=1 \\
\displaystyle \frac{\operatorname{iG}(k)}{\log_2 k} & k \geq 2
\end{cases}$$|
|Ideal DCG|$$\operatorname{iDCG}(k) = \sum_{i=1}^k \operatorname{iDG}(k)$$|
|Normalised DCG|$$\operatorname{nDCG}(k) = \frac{\operatorname{DCG}(k)}{\operatorname{iDCG}(k)}$$|

### Text Retrieval Conferences (TREC)

- Main IR evaluation compaign
- To search a set of docs of given genre and domain
- Formed of a set of tracks, each track has a set of search tasks

#### Collection

- Cover most of the domains in life
- A set of hundreds of thousands of docs
- Format:
```xml
<DOC>
    <DOCNO>1234</DOCNO>
    <TEXT>Plain text of the document</TEXT>
</DOC>
```

#### Topic

- Query sets provided for each collection
- Generated by experts
- Format:
```xml
<num>189</num>
<title>the query text</title>
<desc>description of what is meant by the query</desc>
<narr>what should be considered relevant</narr>
```

### Pooling

1. Each system submit top 1000 docs per topic
1. For each system, judge top 100 docs by
    - single poll, remove duplicates, random rank
    - judged by the person developing the topic
1. Treat unevaluated docs as irrelevant
1. Compute MAP down to 1000 docs


- Large number of reasonable systems
- Systems must not all do the same thing

### Cohen's kappa

$$\varkappa = \frac{P(A)-P(E)}{1-P(E)}$$

- $P(A) = P(J_1=1, J_2=1) + P(J_1=0, J_2=0)$ (probability of judges agree)
- $P(E) = P(J_1=1)P(J_2=1) + P(J_1=0)P(J_2=0)$ (agreement by chance)

$$\varkappa \in \begin{cases}
\{1\} & \text{total agreement} \\
(0.8, 1) & \text{good agreement} \\
(0.67, 0.8] & \text{'fair' agreement} \\
(0, 0.67) & \text{data providing a suspicious basis for an evaluation} \\
\{0\} & \text{chance agreement} \\
[-1,0) & \text{worse than random}
\end{cases}$$

### A/B testing

1. Have most users use old system
1. Divert a small proportion of traffic to the new system
1. Evaluate with automatic measures
1. Use significance test

**Paired t-test (one-sided):**

$$t = \frac{\bar d \cdot \sqrt{n}}{\operatorname{std}(d)} \sim t_{n-1}$$

## Query Expansion

### Automatic thesaurus

**Co-occurence:**

$$\mathbf C = \mathbf{AA}^\top$$

- Pros: unsupervised
- Cons: 
    - related words more than real synonyms
    - computationally expensive
    
**Parallel corpus:**

- Sets of two parallel corpus in two different languages (source and target lang)
- Main training resource for machine translation systems
- Improvement not consistent due to lack of context

### Relevance feedback

- Users give feedback to IR system (relevant/irrelevant)


**Authors:** s1680642

**Licensing:** <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.